In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import os
import pandas as pd
import matplotlib.pyplot as plt
import utils

from PIL import Image, ImageDraw
from sklearn.metrics import roc_auc_score, roc_curve, auc
from tqdm import tqdm

from __future__ import print_function
from __future__ import division

import time
import copy

In [ ]:
#THIS CODE IS SAMPLED TO EVALUTATE THE DENSENET NETWORK - change as needed if evaluating another network

#Define data and model directories
data_dir = '/content/drive/My Drive/DATA_DIR/'
model_dir = '/content/drive/My Drive/Full_Data/DenseNet_v1/classification_model_densenet.pt'
save_folder = '/content/drive/My Drive/Eval on Test Data Results/DenseNet/'

# Number of classes in the dataset
num_classes = 2

#input size of images going into model
input_size = 800

In [ ]:
labels_test = pd.read_csv(data_dir + 'test.csv', na_filter=False)

img_class_dict_test = dict(zip(labels_test.image_name, labels_test.annotation))

In [ ]:
#original download test.csv file had /test/ infront of all image names this box was used to change this to just the image name

# image_list = []

# for i in range(1000):

#     image_string = labels_test.image_path[i][5:]
#     image_list.append(image_string)


# test_reformatted = pd.DataFrame({'image_name': image_list, 'annotation': labels_test.annotation})

# test_reformatted.to_csv(data_dir+'test.csv',columns=['image_name', 'annotation'], sep=',', index=None)

In [ ]:
class ForeignObjectDataset(object):
    
    def __init__(self, datafolder, datatype='train', transform = True, labels_dict={}):
        self.datafolder = datafolder
        self.datatype = datatype
        self.labels_dict = labels_dict
        self.image_files_list = [s for s in sorted(os.listdir(datafolder)) if s in labels_dict.keys()]
        self.transform = transform
        self.annotations = [labels_dict[i] for i in self.image_files_list]
            
    def __getitem__(self, idx):
        # load images 
        img_name = self.image_files_list[idx]
        img_path = os.path.join(self.datafolder, img_name)
        img = Image.open(img_path).convert("RGB")
        width, height = img.size[0],img.size[1]  

        if self.datatype == 'test':

            if self.labels_dict[img_name] == '':
                label = 0
            else:
                label = 1
            
            if self.transform is not None:
                img = self.transform(img)

            return img, label

        

    def __len__(self):
        return len(self.image_files_list)

In [ ]:
# Just normalization for test data

data_transforms = {
    'test': transforms.Compose([
        transforms.Resize((input_size,input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

dataset_test = ForeignObjectDataset(datafolder= data_dir + 'test/', datatype='test', transform=data_transforms['test'], labels_dict=img_class_dict_test)


In [ ]:
data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle= False, num_workers=4,
    collate_fn=utils.collate_fn)

# Create training and validation dataloaders
dataloaders_dict = {'test': data_loader_test}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# #VGG
# def _get_model(num_classes,feature_extract):    
#     model = models.vgg11_bn(pretrained=True)    
#     num_ftrs = model.classifier[6].in_features    
#     model.classifier[6] = nn.Linear(num_ftrs,num_classes)    
#     return model

# #Squeezenet
# def _get_model(num_classes,feature_extract):
#     model = models.squeezenet1_0(pretrained=True)
#     model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
#     model.num_classes = num_classes
#     return model

# #ResNet101
# def _get_model(num_classes,feature_extract):
#     model = models.resnet101(pretrained = True)
#     num_ftrs = model.fc.in_features
#     model.fc = nn.Linear(num_ftrs, num_classes)
#     return model

# #ResNet50
# def _get_model(num_classes,feature_extract):
#     model = models.resnet50(pretrained = True)
#     num_ftrs = model.fc.in_features
#     model.fc = nn.Linear(num_ftrs, num_classes)
#     return model

# #InceptionV3
# def _get_model(num_classes,feature_extract):    
#     model = models.inception_v3(pretrained=True)       
#     # Handle the auxilary net    
#     num_ftrs = model.AuxLogits.fc.in_features    
#     model.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)    
#     # Handle the primary net    
#     num_ftrs = model.fc.in_features    
#     model.fc = nn.Linear(num_ftrs,num_classes)    
#     return model

#DenseNet
def _get_model(num_classes,feature_extract):    
    model = models.densenet121(pretrained=True)   
    num_ftrs = model.classifier.in_features    
    model.classifier = nn.Linear(num_ftrs, num_classes)
    return model

In [ ]:
model = _get_model(num_classes,feature_extract)
model.to(device)
model.load_state_dict(torch.load(model_dir))

with torch.no_grad():
  model.eval()

  preds_prob = []
  labels = []
  locs = []

  for image, label in tqdm(dataloaders_dict['test']):
      
      image = torch.stack(image)
      image = image.to(device)

      labels.append(label[-1])
      
      outputs = model(image)
      outputs = torch.nn.Softmax(dim = 1)(outputs)

      preds_prob.append(outputs[0][1])

preds_prob=torch.Tensor(preds_prob)
preds_prob=preds_prob.tolist()

In [ ]:
pred = []
for i in range(len(preds_prob)):
  if (preds_prob[i] >= .5):
    pred.append(1)
  else:
    pred.append(0)

gt = labels_test.annotation.astype(bool).astype(float).values

plt.plot(gt, preds_prob,'.', Color = [0,0,0,0.1])
plt.ylim((0,1))

In [ ]:
acc = (pred == gt).mean()
fpr, tpr, _ = roc_curve(gt, preds_prob)
roc_auc = auc(fpr, tpr)

In [ ]:
fig, ax = plt.subplots(
    subplot_kw=dict(xlim=[0, 1], ylim=[0, 1], aspect='equal'),
    figsize=(6, 6)
)
ax.plot(fpr, tpr, label=f'ACC: {acc:.03}\nAUC: {roc_auc:.03}')
_ = ax.legend(loc="lower right")
_ = ax.set_title('ROC curve')
ax.grid(linestyle='dashed')

In [ ]:
cls_res = pd.DataFrame({'image_name': dataset_test.image_files_list, 'prediction': preds_prob})
cls_res.to_csv(save_folder + 'test_classification.csv', columns=['image_name', 'prediction'], sep=',', index=None)
print('classification.csv generated.')